(tutorial_debugging_label)=

# Debug your models efficiently

This tutorial gives insights on how you can debug tespy models. The user
interface of the current implementation might still need some refinement, so
you are invited to raise issues in the github repository. We will change it
based on the feedback. The outputs shown here are based on the following
version of tespy:

In [ ]:
from tespy import __version__
__version__

## Simple model debugging

This tutorial will show a couple of things

1. How to extract the variables of the problem

   - before presolving step
   - after presolving step and identify the presolved variables

2. How to extract the applied equations of the problem

   - before presolving step
   - after presolving step and identify the presolved equations

3. How to read and fix the errors that are raised during presolving
4. How to debug a model in case of linear dependency by inspecting the
   error message, incidence matrix and Jacobian
5. How to interpret/deal with a couple of warnings/errors that might pop up
   during postprocessing

### Model overview

The model we implement is a very simple heat pump model, just as implemented
in the introductory {ref}`basics_heat_pump_label` tutorial.

```{image} /_static/images/basics/heat_pump.svg
:align: center
:class: only-light
```

```{image} /_static/images/basics/heat_pump_darkmode.svg
:align: center
:class: only-dark
```

### Model code

In [ ]:
from tespy.components import CycleCloser, SimpleHeatExchanger, Compressor, Valve, Motor, PowerSource
from tespy.connections import Connection, PowerConnection
from tespy.networks import Network

In [ ]:
nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar",
    power="kW",
    heat="kW",
    enthalpy="kJ/kg"
)

In [ ]:
grid = PowerSource("grid")
motor = Motor("motor")

cc = CycleCloser("cycle closer")
valve = Valve("valve")
evaporator = SimpleHeatExchanger("evaporator")
compressor = Compressor("compressor")
condenser = SimpleHeatExchanger("condenser")

c1 = Connection(cc, "out1", evaporator, "in1", label="c1")
c2 = Connection(evaporator, "out1", compressor, "in1", label="c2")
c3 = Connection(compressor, "out1", condenser, "in1", label="c3")
c4 = Connection(condenser, "out1", valve, "in1", label="c4")
c0 = Connection(valve, "out1", cc, "in1", label="c0")


nw.add_conns(c1, c2, c3, c4, c0)

e1 = PowerConnection(grid, "power", motor, "power_in", label="e1")
e2 = PowerConnection(motor, "power_out", compressor, "power", label="e2")

nw.add_conns(e1, e2)

### Debug the model

#### Variable and equation identification

With nothing specified and trying to solve we will get an information, that
the network is lacking fluid information.

In [ ]:
nw.solve("design", init_only=True)

Then let's specify the fluid and try to preprocess the network. With
{code}`solve_determination` we can check, how many parameters are specified and
how many are missing. When running with {code}`init_only` no error is raised,
so we can use that starting point in an interactive python environment to get
started with debugging.

In [ ]:
c1.set_attr(fluid={"R290": 1})
nw.solve("design", init_only=True)
# this would be executed as next step for actual solve call
nw.solve_determination()

Now we can check the following information:

- Which are the original variables of our model
- Which of those variables have already been determined by the presolving
- Which ones are the actual variables, that the model has to solve and which
  original variables of the model these variables represent


The original variables:

In [ ]:
nw.get_variables_before_presolve()

The variables solved already by the presolving step:

In [ ]:
nw.get_presolved_variables()

The actual variables of the problem as a dictionary:

- keys: tuple with variable number as first element, variable type as second
  element (mass flow, pressure, enthalpy, fluid, ...)
- values: list of the original variables this variable represents. The list
  again contains tuples with

  - the label of the component/connection from which the variable originated
  - the type of variable as second element

In [ ]:
nw.get_variables()

We can also check which equations of the model have been presolved in order to
retrieve the dependencies between the variables. E.g. the mass flow variable
just before represents all the mass flows in this model. We can see, that the
{code}`mass_flow_constraints` have been solved for all components. The
equations indicated here can be inspected in the tables of the documentation on
the {ref}`components <modules_components_label>` and 
{ref}`connections <modules_connections_label>`.

In [ ]:
nw.get_presolved_equations()

There is not yet an easy way to identify which variable was presolved by which.
Next to the presolved equations we can also inspect, which equations are 
present in the actual model that needs to be solved iteratively.

In [ ]:
nw.get_equations()

With the equations we can also extract the variables these depend on.

In [ ]:
nw.get_equations_with_dependents()

#### Impose parameters and check again

Now let's impose a couple of boundary conditions:

- No pressure drop in heat exchanges
- Compressor efficiency
- Motor efficiency

In [ ]:
condenser.set_attr(dp=0)
evaporator.set_attr(dp=0)
compressor.set_attr(eta_s=0.8)
motor.set_attr(eta=0.97)
nw.solve("design", init_only=True)

Again, we can inspect, which variables have been presolved now. It does not
change, because we did not impose any boundary conditions, where any of the
variables can be directly determined from.

In [ ]:
nw.get_presolved_variables()

But if we check the actual variables of the system, we see that the number has
been reduced. The two energy flows are now mapped to a single variable, and
the pressure values before and after the heat exchangers have been also mapped
to a single variable respectively.

In [ ]:
nw.get_variables()

The reason for that can be seen in the presolved equations, where now we have
three additional entries.

In [ ]:
nw.get_presolved_equations()

And we also get one more equation in our model equations, that needs to be 
solved numerically: the compressor efficiency.

In [ ]:
nw.get_equations_with_dependents()

Let's add more boundary conditions, because we are still missing a couple:

- evaporation temperature level and superheating

In [ ]:
c2.set_attr(T_dew=10, td_dew=10)
nw.solve("design", init_only=True)

Now we can see that the number of variables has been reduced by two. The reason
for this is, that the presolver was able to identify pressure and enthalpy
at the compressor inlet with the given boundary conditions.

In [ ]:
nw.get_variables()

Since these two variables have now been presolved, the equation of the 
compressor has less dependents, as it is not necessary to solve for the
respective variables anymore. 

In [ ]:
nw.get_equations_with_dependents()

We are still missing 3 equations as we have 5 variables in the problem and only
2 equations at the moment, so let's add the missing specifications:

- electrical power input
- condensing temperature level and subcooling

In [ ]:
c4.set_attr(T_bubble=60, td_bubble=0)
e1.set_attr(E=100)  # 100 kW

In [ ]:
nw.solve("design")

#### Handle errors during presolving

Some errors can occur during presolving, for example:

You specify a linear change of specific variable while specifying both values
simultaneously. In this case, the error message directly tells you which
variables are linear dependent and that you specified more than a single value
in that set (points to the labels of the connections/components).

In [ ]:
e2.set_attr(E=97)
nw.solve("design")

We can see the same problem if we were to specify compressor pressure ratio:
The compressor inlet pressure is determined from the compressor inlet state,
the condenser outlet pressure is determined from the condenser outlet state and
the condenser pressure drop is specified. By that also the compressor outlet
pressure is known and you cannot specify the outlet pressure.

In [ ]:
e2.set_attr(E=None)
compressor.set_attr(pr=4)
nw.solve("design", init_only=True)

You can also think of creating a circular dependency. For example, if you
specify a relationship of mass flow in front and behind the cycle closer (or
if you were to remove the cycle closer). Then the mass flow would form a
circular dependency. As output of the error message you get the variables which
are part of the circular dependency and the equations responsible for that.


In [ ]:
compressor.set_attr(pr=None)


from tespy.connections import Ref

c1.set_attr(m=Ref(c0, 1, 0))
nw.solve("design", init_only=True)

A last error that might occur is specification of properties that determine the
same variable, e.g. the c2 pressure as the evaporation pressure is already
determined from the saturation temperature and superheating.

In [ ]:
c1.set_attr(m=None)
c2.set_attr(p=10)  # p has already been set!
nw.solve("design")

In [ ]:
c2.set_attr(p=None)

#### Inspect reasons for linear dependency

You can also inspect the network after crashing due to a linear dependency in
the Jacobian. For this, we will construct a case where, we get this exact
issue:

- We fix heat output of condenser (having fixed motor electrical power already)
- no specification of evaporator delta p

In [ ]:
condenser.set_attr(Q=-350)
evaporator.set_attr(dp=None)
nw.solve("design")

The solver now tells us:

- The problem is likely a problem in the setup
- The reason is that a variable is not associated with any equation

We can then retrieve, which unknowns the variable represents to understand,
the issue behind it.

In [ ]:
nw.get_variables()

And we can identify, that there is no equation, that depends on that variable:

In [ ]:
nw.get_equations_with_dependents()

If the problem is not a setup based problem but a numerical one due to
partial derivatives in the Jacobian becoming zero where they should not, it is
also possible to inspect the issue. For this we will change the model setup.
Just a normal HeatExchanger is sufficient for that:

In [ ]:
from tespy.components import Source, Sink, HeatExchanger


nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)

so1 = Source("source 1")
so2 = Source("source 2")

si1 = Sink("sink 1")
si2 = Sink("sink 2")

heatex = HeatExchanger("heatexchanger")

c1 = Connection(so1, "out1", heatex, "in1", label="c1")
c2 = Connection(heatex, "out1", si1, "in1", label="c2")
d1 = Connection(so2, "out1", heatex, "in2", label="d1")
d2 = Connection(heatex, "out2", si2, "in1", label="d2")

nw.add_conns(c1, c2, d1, d2)

Now we could make a specification that is impossible but hard to catch as being
a setup problem: We set a minimum terminal temperature difference of 25 K but
at the same time fix the temperature at hot side outlet and cold side inlet
(leading to a temperature difference of 20 K).

In [ ]:
c1.set_attr(fluid={"air": 1}, T=200, p=1, m=5)
c2.set_attr(T=110)
d1.set_attr(fluid={"water": 1}, T=90, p=1)
heatex.set_attr(dp1=0, dp2=0, ttd_min=25)

In [ ]:
nw.solve("design")

We can see:

- The solver tells us potentially numerical issue (not always correct!).
- The heat exchanger ttd_min function should depend on variable (0, "h")
  representing "h" of connection "d2".
- There is a zero in the Jacobian of that function towards that number while
  there should be a non-zero entry.
- An extra piece of information: all entries in the Jacobian of the equation
  became zero.

Next we can also extract the incidence matrix and the Jacobian to identify
issue if we want. For that we have a look at the equations and their dependents
first.

In [ ]:
nw.get_equations_with_dependents()

Then we can use the incidence_matrix and the Jacobian. Here we see for the
problematic equation number (1) and variable number (0):

- The incidence matrix indicates which entries (rows = equations, columns =
  variables) of the Jacobian are supposed to be zero or non-zero.
- The actual values of the Jacobian.
- And, that a non-zero is expected for (1, 0) but there is a zero instead.

In [ ]:
nw._incidence_matrix_dense

In [ ]:
nw.jacobian